# Discharge Notes and Readmission Rates: EDA

This notebook is for the initial exploratory data analysis of the dataset. Readmission is defined as any admissions that had a pervious admissions regardless of the time difference.

## Overview

## Business Understanding

## Data Understanding

## Import

Import all the relevant libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re

In [1]:
#set the pandas display column width as max to read in all the text
#pd.set_option('display.max_colwidth', None)

NameError: name 'pd' is not defined

## Functions

This section is for any functions created for the analysis.

In [3]:
def group_categories(df, category):
    '''
    some of the datasets have multiple rows for diagnosis or ICD code per subject and admission ID
    in order to onehotencode it later, this function aggregates all the codes into a list, and then
    convert the list into a string separate by a comma
    
    the function takes in the dataframe which should only contain three columns: subject_id, hadm_id,
    and the category of interest; and the string for the column name of the cateogry of interest
    '''
    #group by the subject and admission ID
    group = df.groupby(['SUBJECT_ID', 'HADM_ID'])
    #create a list of all the values associated with the subject and admission ID
    result = group.agg({category:lambda x:list(set(x.tolist()))})
    result.reset_index(inplace=True)
    #convert the list into a string, separated by commas
    result[category] = [', '.join(map(str, l)) for l in result[category]]
    return result

## Exploratory Data Analysis (EDA)

In this section, I focused on reviewing the datasets I am using for this project.

### Admission Data

The ADMISSIONS dataset has the information on the demographics of the patient. 

In [4]:
#read in the dataset
admit_df = pd.read_csv('data/ADMISSIONS.csv.gz', compression='gzip')

In [5]:
#review the columns in the dataset
admit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID                58976 non-null  int64 
 1   SUBJECT_ID            58976 non-null  int64 
 2   HADM_ID               58976 non-null  int64 
 3   ADMITTIME             58976 non-null  object
 4   DISCHTIME             58976 non-null  object
 5   DEATHTIME             5854 non-null   object
 6   ADMISSION_TYPE        58976 non-null  object
 7   ADMISSION_LOCATION    58976 non-null  object
 8   DISCHARGE_LOCATION    58976 non-null  object
 9   INSURANCE             58976 non-null  object
 10  LANGUAGE              33644 non-null  object
 11  RELIGION              58518 non-null  object
 12  MARITAL_STATUS        48848 non-null  object
 13  ETHNICITY             58976 non-null  object
 14  EDREGTIME             30877 non-null  object
 15  EDOUTTIME             30877 non-null

In [6]:
#review the first 5 entires in the dataset
admit_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [7]:
#check what admission types are included
admit_df['ADMISSION_TYPE'].value_counts()

EMERGENCY    42071
NEWBORN       7863
ELECTIVE      7706
URGENT        1336
Name: ADMISSION_TYPE, dtype: int64

In [8]:
#drop any irrelevant columns
columns_to_drop = ['DEATHTIME','EDREGTIME','EDOUTTIME','HOSPITAL_EXPIRE_FLAG']
admit_df.drop(columns_to_drop, axis=1, inplace=True)

### Chart Notes

The NOTEEVENTS dataset has the chart notes, including discharge notes, per patient and admissions.

In [9]:
#read in the dataset
notes_df = pd.read_csv('data/NOTEEVENTS.csv.gz', compression='gzip')

/Users/arthur/opt/anaconda3/envs/main_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
#review the columns in the dataset
notes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


In [11]:
#review the first 5 entires in the dataset
notes_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [12]:
#the column ISERROR, if the value is 1, indicates that a physician indicated that the notes are wrong
#drop any columns that do not have an error
notes_df = notes_df[notes_df['ISERROR'] != 1]
#isolate the discharge notes
notes_df = notes_df[notes_df['CATEGORY'] == 'Discharge summary']

In [13]:
#drop any irrelevant columns
columns_to_drop = ['ROW_ID','CHARTDATE','CHARTTIME','STORETIME','CATEGORY','DESCRIPTION','CGID','ISERROR']
notes_df.drop(columns_to_drop, axis=1, inplace=True)

In [14]:
#review the edited dataframe
notes_df.head()

,SUBJECT_ID,HADM_ID,TEXT
0,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...
1,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...
2,13702,167118.0,Admission Date: [**2119-5-4**] D...
3,13702,196489.0,Admission Date: [**2124-7-21**] ...
4,26880,135453.0,Admission Date: [**2162-3-3**] D...


In [15]:
notes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59652 entries, 0 to 60415
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SUBJECT_ID  59652 non-null  int64  
 1   HADM_ID     59652 non-null  float64
 2   TEXT        59652 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.8+ MB


In [16]:
#it seems like there are multiple discharge notes per subject and admission IDs
notes_df.duplicated(subset=['SUBJECT_ID','HADM_ID'], keep=False)

0         True
1        False
2        False
3        False
4        False
         ...  
60411     True
60412     True
60413     True
60414     True
60415     True
Length: 59652, dtype: bool

In [17]:
#combine any texts that have the same SUBJECT_ID and HADM_ID
notes_df = group_categories(notes_df, 'TEXT')

In [18]:
notes_df

,SUBJECT_ID,HADM_ID,TEXT
0,3,145834.0,Admission Date: [**2101-10-20**] Discharg...
1,4,185777.0,Admission Date: [**2191-3-16**] Discharge...
2,6,107064.0,Admission Date: [**2175-5-30**] Dischar...
3,9,150750.0,"Name: [**Known lastname 10050**], [**Known fi..."
4,10,184167.0,Admission Date: [**2103-6-28**] Dischar...
...,...,...,...
52721,99985,176670.0,Admission Date: [**2181-1-27**] ...
52722,99991,151118.0,Admission Date: [**2184-12-24**] ...
52723,99992,197084.0,Admission Date: [**2144-7-25**] ...
52724,99995,137810.0,Admission Date: [**2147-2-8**] D...


### Additional Patient Data

The PATIENT dataset contains further information of the patient include date of birth and gender

In [19]:
#read in the dataset
patients_df = pd.read_csv('data/PATIENTS.csv.gz', compression='gzip')

In [20]:
#review the columns in the dataset
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46520 entries, 0 to 46519
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       46520 non-null  int64 
 1   SUBJECT_ID   46520 non-null  int64 
 2   GENDER       46520 non-null  object
 3   DOB          46520 non-null  object
 4   DOD          15759 non-null  object
 5   DOD_HOSP     9974 non-null   object
 6   DOD_SSN      13378 non-null  object
 7   EXPIRE_FLAG  46520 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 2.8+ MB


In [21]:
#review the first 5 entires in the dataset
patients_df.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [22]:
#isolate the relevant columns
patients_df = patients_df[['SUBJECT_ID', 'GENDER', 'DOB']]

### CPT Codes

The CPTEVENTS dataset contains the CPT (Current Procedural Terminology ) codes are five digit number assigned to every service a medical practitioner provided to a patient.

In [ ]:
#read in the dataset
cpt_events_df = pd.read_csv('data/CPTEVENTS.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
cpt_events_df.info()

In [ ]:
#review the first 5 entires in the dataset
cpt_events_df.head()

In [ ]:
#isolate the relevant columns
cpt_events_df = cpt_events_df[['SUBJECT_ID', 'HADM_ID', 'CPT_CD']]

### Diagnosis ICD Codes

The DIAGNOSES dataset contains all diagnoses per subject and admission IDs. The column SEQ_NUM contains the order in which the ICD diagnoses relate to the patient.

In [ ]:
#read in the dataset
diagnoses_icd_df = pd.read_csv('data/DIAGNOSES_ICD.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
diagnoses_icd_df.info()

In [ ]:
#review the first 5 entires in the dataset
diagnoses_icd_df.head()

In [ ]:
#isolate the relevant columns
diagnoses_icd_df = diagnoses_icd_df[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]

### DRG Codes

The DRGCODES contains the DRG (diagnosis-related group) codes, which is a patient classification system that standardizes prospective payment.

In [ ]:
#read in the dataset
drg_codes_df = pd.read_csv('data/DRGCODES.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
drg_codes_df.info()

In [ ]:
#review the first 5 entires in the dataset
drg_codes_df.head()

In [ ]:
#isolate the relevant columns
drg_codes_df = drg_codes_df[['SUBJECT_ID','HADM_ID','DRG_CODE']]

### Procedure ICD Codes

The PROCEDURES_ICD dataset contains the procedure ICD codes per subject and patient IDs.

In [ ]:
#read in the dataset
proced_icd_df = pd.read_csv('data/PROCEDURES_ICD.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
proced_icd_df.info()

In [ ]:
#review the first 5 entires in the dataset
proced_icd_df.head()

In [ ]:
#isolate the relevant columns
proced_icd_df = proced_icd_df[['SUBJECT_ID','HADM_ID','ICD9_CODE']]

## Removing Last Duplicate Admissions

Since some patients are readmitted multiple times at the hospital, I wanted to remove the last visit they had to the hospital since I cannot use the discharge note for the last visit since there would be no subsequent readmissions

In [39]:
#sort the dates in ascending order, so that if we were to delete the last duplicate, it would be the latest admissions
admit_df.sort_values('ADMITTIME', inplace=True)

In [40]:
#create a dataframe on the number of times a patient was admitted
dupli_admit_df = pd.DataFrame(admit_df.groupby('SUBJECT_ID')['HADM_ID'].count())
dupli_admit_df.rename(columns={'HADM_ID': 'ADMIT_COUNTS'}, inplace=True)

In [41]:
#print out how many patients were admitted once vs admitted multiple times
print(dupli_admit_df[dupli_admit_df['ADMIT_COUNTS']==1].count())
print(dupli_admit_df[dupli_admit_df['ADMIT_COUNTS']>1].count())

ADMIT_COUNTS    38983
dtype: int64
ADMIT_COUNTS    7537
dtype: int64


We can see here that there is clearly a class imbalance that we will have to take into account when modeling.

In [42]:
#merged the admission counts back into the main dataframe
new_df = admit_df.merge(dupli_admit_df,on='SUBJECT_ID')

#separate out the people who were never readmitted
mod_admit_df = new_df[new_df['ADMIT_COUNTS']==1]
mod_admit_df['ADMISSION_TYPE'].value_counts()

EMERGENCY    25417
NEWBORN       7603
ELECTIVE      5066
URGENT         897
Name: ADMISSION_TYPE, dtype: int64

In [43]:
#separate out the people who were readmitted multiple times
duplicate_df = new_df[new_df['ADMIT_COUNTS']>1]

#create a copy of the dataframe with the duplicates
df1 = duplicate_df.copy()

In [44]:
#create a list of the unique number of times people were readmitted
admit_counts = sorted(df1['ADMIT_COUNTS'].unique())

In [45]:
#for every admission counts
for i in admit_counts:
    #find the patients associated with that admission count
    df2 = df1[df1['ADMIT_COUNTS']==i]
    
    #this while loop is to iterate the dataframe and retrieving the first duplicate of the main dataframe
    #and then the main dataframe becomes the deleted out duplicates, this process is repeated until the 
    #admin counts reaches 1 at 'i -=1'
    while i > 1:
        #this dataframe holds the duplicates
        df3 = df2.loc[df2.duplicated(subset='SUBJECT_ID')]
        #this datafame holds the de-duped dataframe
        df4 = df2.drop_duplicates('SUBJECT_ID')
        #add the deduped dataframe to the main dataframe
        mod_admit_df = pd.concat([mod_admit_df, df4])
        df2 = df3
        i -= 1

In [46]:
#review the modified DataFrame to see if it worked
mod_admit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51439 entries, 0 to 47700
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID                51439 non-null  int64 
 1   SUBJECT_ID            51439 non-null  int64 
 2   HADM_ID               51439 non-null  int64 
 3   ADMITTIME             51439 non-null  object
 4   DISCHTIME             51439 non-null  object
 5   ADMISSION_TYPE        51439 non-null  object
 6   ADMISSION_LOCATION    51439 non-null  object
 7   DISCHARGE_LOCATION    51439 non-null  object
 8   INSURANCE             51439 non-null  object
 9   LANGUAGE              28220 non-null  object
 10  RELIGION              50994 non-null  object
 11  MARITAL_STATUS        41633 non-null  object
 12  ETHNICITY             51439 non-null  object
 13  DIAGNOSIS             51417 non-null  object
 14  HAS_CHARTEVENTS_DATA  51439 non-null  int64 
 15  ADMIT_COUNTS          51439 non-null

In [47]:
mod_admit_df[mod_admit_df['ADMIT_COUNTS']==3].sort_values('SUBJECT_ID').head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,ADMIT_COUNTS
18056,37,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CHEST PAIN/SHORTNESS OF BREATH,1,3
18055,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,3
8491,112,107,191941,2115-02-20 17:41:00,2115-02-21 16:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,AV FISTULA;ANGIOPLASTY,1,3
8492,113,107,182383,2121-11-30 19:24:00,2121-12-05 14:18:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,SEPARATED,HISPANIC OR LATINO,HYPERKALEMIA;CONGESTIVE HEART FAILURE,1,3
18407,186,138,188284,2133-12-21 23:52:00,2134-01-11 13:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,HEMATURIA;RENAL MASS,1,3
18406,185,138,108120,2131-10-31 08:00:00,2131-11-06 12:54:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,MARRIED,WHITE,AORTIC STENOSIS\ AORTIC VALVE REPLACEMENT/SDA,1,3
10164,205,154,162891,2118-04-05 18:11:00,2118-04-11 14:21:00,URGENT,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,NaN,EPISCOPALIAN,MARRIED,WHITE,UNSTABLE ANGINA\CATH WITH BRACHY S/B,1,3
10163,204,154,111735,2117-12-29 21:36:00,2118-01-01 14:27:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,EPISCOPALIAN,MARRIED,WHITE,ACUTE MYOCARDIAL INFARCTION,1,3
20287,308,236,182562,2135-05-26 11:28:00,2135-05-28 16:46:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,PATIENT DECLINED TO ANSWER,POST OP BLEEDING,1,3
20286,307,236,151459,2134-10-04 01:19:00,2134-10-15 16:00:00,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,CATHOLIC,MARRIED,PATIENT DECLINED TO ANSWER,END STAGE LIVER DISEASE,1,3


As we can see the patients who were admitted three times showed up twice in our dataframe, meaning that one duplicate has successfully been removed. Below, we pull subject ID 36 as an example from the original dataframe to see that the last admission visit was removed.

In [48]:
#double checking that the last admissions time was the one that was removed
admit_df[admit_df['SUBJECT_ID'] == 36]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA
15,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1
16,37,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CHEST PAIN/SHORTNESS OF BREATH,1
17,38,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,VENTRAL HERNIA/SDA,1


In [49]:
#convert all the admission counts to 1 (readmission occurred after this admissions) or 0 (no readmission)
mod_admit_df.loc[mod_admit_df['ADMIT_COUNTS'] == 1, 'ADMIT_COUNTS'] = 0
mod_admit_df.loc[mod_admit_df['ADMIT_COUNTS'] > 1, 'ADMIT_COUNTS'] = 1
mod_admit_df.rename(columns={'ADMIT_COUNTS': 'READMISSION'}, inplace=True)

## Grouping Codes by Subject and Admission IDs

Some of the datasets need to be grouped by subject and admission IDs such as the diagonsis and ICD codes, since each admissions could have multiple codes.

In [50]:
#group the CPT dataset
cpt_events_df = group_categories(cpt_events_df, 'CPT_CD')

In [51]:
#group the diagnoses dataset
diagnoses_icd_df = group_categories(diagnoses_icd_df, 'ICD9_CODE')
diagnoses_icd_df.rename(columns={'ICD9_CODE': 'DIAG_ICD9_CODE'}, inplace=True)

In [52]:
#group the DRG dataset
drg_codes_df = group_categories(drg_codes_df, 'DRG_CODE')

In [53]:
#group the procedures dataset
proced_icd_df = group_categories(proced_icd_df, 'ICD9_CODE')
proced_icd_df.rename(columns={'ICD9_CODE': 'PROCED_ICD9_CODE'}, inplace=True)

## Merging Dataframes

Here, I merged some of the datasets based on the subject and admissions IDs.

The modified admissions and PATIENTS datasets can be merged using the subject ID. The PATIENTS dataset has the date of birth (DOB) of the patient, which can be subtracted by the discharge date to get the age of the patient. However, patients who are over 89 years old had their DOB shifted to 300 years after the admission date to comply with HIPAA regulations.

In [54]:
#merge the modified admissions dataset with the PATIENTS dataset
mod_admit_df = mod_admit_df.merge(patients_df, on='SUBJECT_ID')

In [55]:
#obtain the year, month, and date of the discharge time
mod_admit_df['DISCHTIME'] = mod_admit_df['DISCHTIME'].str[:10]
mod_admit_df['DOB'] = mod_admit_df['DOB'].str[:10]

In [56]:
#review the modified discharge time
mod_admit_df['DISCHTIME']

0        2100-06-09
1        2100-06-19
2        2100-06-27
3        2100-06-17
4        2100-06-30
            ...    
51434    2187-03-05
51435    2187-04-13
51436    2187-07-14
51437    2187-07-27
51438    2187-08-02
Name: DISCHTIME, Length: 51439, dtype: object

In [57]:
#convert the discharge time and the date of birth into datetime
mod_admit_df['DISCHTIME'] = pd.to_datetime(mod_admit_df['DISCHTIME'], format="%Y/%m/%d")
mod_admit_df['DOB'] = pd.to_datetime(mod_admit_df['DOB'], format="%Y/%m/%d")

In [58]:
#convert the discharge time and the date of birth to time series
ts_disch = mod_admit_df['DISCHTIME'].dt.to_pydatetime()
ts_dob = mod_admit_df['DOB'].dt.to_pydatetime()

In [59]:
#subtract the dischrage time by the date of birth, and divide by 365 to get years
age_df = pd.DataFrame((ts_disch-ts_dob)//365)

In [60]:
#retrieve the age, but note that because the structure of the datetime data, dt.days will return the int value of whatever is in the columns. Since I
#already divided by 365, when I use dt.days, I am actually getting the years
mod_admit_df['AGE'] = age_df[0].dt.days

In [61]:
#check the value counts of the age to see how many of the values are 300
mod_admit_df['AGE'].value_counts()

0      7882
300    2077
77      954
78      943
69      942
       ... 
307       7
310       3
308       3
14        1
309       1
Name: AGE, Length: 88, dtype: int64

About 2K admissions were of patients who were older than 89. In order to adjust the number, I assumed that all people ages "300" are 90.

In [62]:
#change anyone with an age of 300 to 90
mod_admit_df.loc[mod_admit_df['AGE'] == 300, "AGE"] = 90

In addition, I merged the CPT, 

In [63]:
#merge the admissions dataframe
main_df = mod_admit_df.merge(notes_df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
main_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMISSION,GENDER,DOB,AGE,TEXT
0,53486,82574,118464,2100-06-07 19:59:00,2100-06-09,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,CATHOLIC,SINGLE,OTHER,HYPOTENSION,1,0,M,2044-04-23,56,Admission Date: [**2100-6-7**] D...
1,35574,29156,161773,2100-06-09 01:39:00,2100-06-19,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,UNOBTAINABLE,MARRIED,WHITE,RESPIRATORY DISTRESS,1,0,M,2028-04-23,72,Admission Date: [**2100-6-9**] D...
2,14753,12001,173927,2100-06-14 04:55:00,2100-06-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,RT BASAL GANGLIA BLEEDING,1,0,F,2028-10-27,71,Admission Date: [**2100-6-14**] Dischar...
3,25723,21081,159656,2100-06-14 12:02:00,2100-06-17,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,SHORTNESS OF BREATH,1,0,F,2067-04-01,33,Admission Date: [**2100-6-14**] Discharge...
4,38932,32096,158366,2100-06-22 03:04:00,2100-06-30,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,"STERNAL FRACTURE,RIB FRACTURE,LACERATIONS",1,0,F,2070-12-13,29,Admission Date: [**2100-6-22**] ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45586,15999,13033,107751,2187-03-01 04:32:00,2187-03-05,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,CATHOLIC,SINGLE,BLACK/AFRICAN AMERICAN,ACUTE CORONARY SYNDROME,1,1,M,2148-04-23,38,Admission Date: [**2187-3-1**] D...
45587,16000,13033,161468,2187-04-05 16:22:00,2187-04-13,EMERGENCY,EMERGENCY ROOM ADMIT,LEFT AGAINST MEDICAL ADVI,Medicare,ENGL,CATHOLIC,SINGLE,BLACK/AFRICAN AMERICAN,ABDOMINAL PAIN;TELEMETRY,1,1,M,2148-04-23,38,Admission Date: [**2187-4-5**] D...
45588,16001,13033,132307,2187-07-11 07:19:00,2187-07-14,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,CATHOLIC,SINGLE,BLACK/AFRICAN AMERICAN,ABDOMINAL PAIN,1,1,M,2148-04-23,39,Admission Date: [**2187-7-11**] ...
45589,16002,13033,183332,2187-07-24 13:29:00,2187-07-27,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,CATHOLIC,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSIVE EMERGENCY,1,1,M,2148-04-23,39,Admission Date: [**2187-7-24**] ...


In [64]:
print("The modified admissions dataset has", str(len(mod_admit_df)),"rows.")
print("The notes dataset has", str(len(notes_df)),"rows.")
print("The merged dataset has", str(len(main_df)),"rows.")

The modified admissions dataset has 51439 rows.
The notes dataset has 52726 rows.
The merged dataset has 45591 rows.


From this inner merge, about 6-7K rows were lost. Below I explore where there are null values after doing an 'outer' merge between the admissions and notes datasets. 

In [65]:
#performing outer merge to see where the null values are
main2_df = mod_admit_df.merge(notes_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
main2_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMISSION,GENDER,DOB,AGE,TEXT
0,53486.0,82574,118464.0,2100-06-07 19:59:00,2100-06-09,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,CATHOLIC,SINGLE,OTHER,HYPOTENSION,1.0,0.0,M,2044-04-23,56.0,Admission Date: [**2100-6-7**] D...
1,35574.0,29156,161773.0,2100-06-09 01:39:00,2100-06-19,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,UNOBTAINABLE,MARRIED,WHITE,RESPIRATORY DISTRESS,1.0,0.0,M,2028-04-23,72.0,Admission Date: [**2100-6-9**] D...
2,14753.0,12001,173927.0,2100-06-14 04:55:00,2100-06-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,RT BASAL GANGLIA BLEEDING,1.0,0.0,F,2028-10-27,71.0,Admission Date: [**2100-6-14**] Dischar...
3,25723.0,21081,159656.0,2100-06-14 12:02:00,2100-06-17,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,SHORTNESS OF BREATH,1.0,0.0,F,2067-04-01,33.0,Admission Date: [**2100-6-14**] Discharge...
4,38932.0,32096,158366.0,2100-06-22 03:04:00,2100-06-30,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,"STERNAL FRACTURE,RIB FRACTURE,LACERATIONS",1.0,0.0,F,2070-12-13,29.0,Admission Date: [**2100-6-22**] ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58569,NaN,99822,195871.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,Admission Date: [**2197-7-3**] D...
58570,NaN,99883,198523.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,Admission Date: [**2132-9-15**] ...
58571,NaN,99897,181057.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,Admission Date: [**2182-7-3**] D...
58572,NaN,99923,192053.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,Admission Date: [**2201-5-15**] ...


In [66]:
#reviewing where there is a null for the text value in the merged file from the notes dataset
null_text_df = main2_df[main2_df['TEXT'].isna()]
null_text_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMISSION,GENDER,DOB,AGE,TEXT
5,35034.0,28711,104132.0,2100-07-02 21:05:00,2100-07-07,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,ASIAN - OTHER,NEWBORN,1.0,0.0,M,2100-07-02,0.0,NaN
7,602.0,476,161042.0,2100-07-05 10:19:00,2100-07-07,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,UNOBTAINABLE,NaN,ASIAN,NEWBORN,1.0,0.0,F,2100-07-05,0.0,NaN
29,33653.0,27526,134575.0,2100-07-29 12:08:00,2100-08-03,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,NaN,PROTESTANT QUAKER,NaN,WHITE,NEWBORN,1.0,0.0,F,2100-07-29,0.0,NaN
34,2959.0,2437,104692.0,2100-08-03 08:19:00,2100-08-05,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,NaN,UNOBTAINABLE,NaN,ASIAN,NEWBORN,1.0,0.0,M,2100-08-03,0.0,NaN
48,3715.0,3085,114188.0,2100-08-13 08:43:00,2100-08-17,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,UNOBTAINABLE,NaN,WHITE,NEWBORN,1.0,0.0,M,2100-08-13,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51326,6162.0,5060,169479.0,2182-10-05 23:38:00,2182-10-06,EMERGENCY,EMERGENCY ROOM ADMIT,LEFT AGAINST MEDICAL ADVI,Medicaid,ENGL,NOT SPECIFIED,SINGLE,WHITE,ETOH INTOXICATION,1.0,1.0,M,2144-09-28,38.0,NaN
51338,14554.0,11861,123134.0,2126-02-27 20:39:00,2126-03-01,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,DIABETIC KETOACIDOSIS,1.0,1.0,F,2105-05-05,20.0,NaN
51339,14555.0,11861,141109.0,2126-03-28 06:19:00,2126-03-30,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,DIABETIC KETOACIDOSIS,1.0,1.0,F,2105-05-05,20.0,NaN
51346,14562.0,11861,189636.0,2127-08-15 13:24:00,2127-08-16,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,DIABETIC KETOACIDOSIS,1.0,1.0,F,2105-05-05,22.0,NaN


It seems like most of the rows that were lost during the merge were because of null discharge notes.

In [67]:
null_text_df['ADMISSION_TYPE'].value_counts()

NEWBORN      4209
EMERGENCY    1344
ELECTIVE      251
URGENT         44
Name: ADMISSION_TYPE, dtype: int64

Based on the admission types, ~4.2K of the rows were newborns and therefore they may not need a discharge summary. I will remove them to explore why the remaining ~1.6K were dropped.

In [68]:
null_text_df = null_text_df[null_text_df['ADMISSION_TYPE'] != 'NEWBORN']
null_text_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMISSION,GENDER,DOB,AGE,TEXT
85,44027.0,53252,189277.0,2100-09-13 16:07:00,2100-09-13,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,NOT SPECIFIED,NaN,WHITE,TAMPONADE,1.0,0.0,M,1800-09-13,90.0,NaN
203,27403.0,22465,105036.0,2101-01-02 15:55:00,2101-01-04,EMERGENCY,EMERGENCY ROOM ADMIT,LEFT AGAINST MEDICAL ADVI,Private,NaN,CATHOLIC,SINGLE,WHITE,S/P 30 FEET FALL,1.0,0.0,M,2067-09-13,33.0,NaN
299,17362.0,14182,118776.0,2101-03-20 23:57:00,2101-03-27,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,WIDOWED,WHITE,RT BASOGANLIA HEMORRHAGE,1.0,0.0,F,2020-01-05,81.0,NaN
300,54921.0,87347,195120.0,2101-03-23 18:15:00,2101-03-25,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CAROTID DISEASE\CAROTID ANGIOGRAM,1.0,0.0,M,2035-12-19,65.0,NaN
342,47568.0,64124,129829.0,2101-04-25 18:02:00,2101-05-04,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,SINGLE,WHITE,ACUTE SUBDURAL HEMATOMA,1.0,0.0,F,2038-11-27,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51326,6162.0,5060,169479.0,2182-10-05 23:38:00,2182-10-06,EMERGENCY,EMERGENCY ROOM ADMIT,LEFT AGAINST MEDICAL ADVI,Medicaid,ENGL,NOT SPECIFIED,SINGLE,WHITE,ETOH INTOXICATION,1.0,1.0,M,2144-09-28,38.0,NaN
51338,14554.0,11861,123134.0,2126-02-27 20:39:00,2126-03-01,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,DIABETIC KETOACIDOSIS,1.0,1.0,F,2105-05-05,20.0,NaN
51339,14555.0,11861,141109.0,2126-03-28 06:19:00,2126-03-30,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,DIABETIC KETOACIDOSIS,1.0,1.0,F,2105-05-05,20.0,NaN
51346,14562.0,11861,189636.0,2127-08-15 13:24:00,2127-08-16,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,DIABETIC KETOACIDOSIS,1.0,1.0,F,2105-05-05,22.0,NaN


In [69]:
null_text_df['DISCHARGE_LOCATION'].value_counts()

HOME                         872
DEAD/EXPIRED                 471
HOME HEALTH CARE             131
LEFT AGAINST MEDICAL ADVI     49
SNF                           37
SHORT TERM HOSPITAL           19
REHAB/DISTINCT PART HOSP      17
DISC-TRAN CANCER/CHLDRN H     15
LONG TERM CARE HOSPITAL       10
DISCH-TRAN TO PSYCH HOSP       5
HOSPICE-HOME                   5
OTHER FACILITY                 4
ICF                            2
HOSPICE-MEDICAL FACILITY       1
HOME WITH HOME IV PROVIDR      1
Name: DISCHARGE_LOCATION, dtype: int64

From the discharge location column, about 500 people were either dead/expired or left against medical advice. This further explains some of the null values in the text columns. The remaining ~1K there may be other reasons, but I am assuming that I can drop these rows for now since there are no discharge notes.

I merged the CPT, diagnosis, procedure, DRG codes datasets into one dataframe.

In [70]:
#merge together the four datasets above
merged_df = cpt_events_df.merge(diagnoses_icd_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df = merged_df.merge(drg_codes_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df = merged_df.merge(proced_icd_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df

,SUBJECT_ID,HADM_ID,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE
0,3,145834,"94002, 94003","41071, 4280, 4275, 0389, 6826, 4254, 5849, 785...",416,"9604, 8964, 966, 9672, 9962, 3893"
1,4,185777,"99232, 99233, 99238, 99223, 99231","5715, 04111, 7907, 2763, 042, 7994, E9317, 136...",489,"8872, 3323, 3893"
2,6,107064,"99232, 99233, 99239, 99255, 99291, 99231","2859, 9972, V1582, 40391, 4440, 2767, 2766, 2753",302,"5569, 9904, 3957, 91, 3806"
3,9,150750,94003,"4280, 2765, 4019, 431, 5070, 5849",14,"9672, 9604"
4,11,194540,"99239, 99233",1913,"2, 211","9925, 113, 9229, 159"
...,...,...,...,...,...,...
58971,99017,103056,NaN,"41071, 4439, 2859, 41401, 3659, 496","1742, 247","66, 40, 47, 3606, 3607"
58972,99088,165352,NaN,"4019, 43889, 72989, 2326","578, 3611","864, 8661"
58973,99487,188460,NaN,"2859, 41401, 4019, 2689, 3669, 41021, 2724, 53...","1742, 247","66, 40, 3722, 45, 3607, 8856"
58974,99650,190854,NaN,"4139, 5363, 36201, 25050, 41401, 5856, 40391, ...","554, 1733","3929, 3995"


In [71]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 0 to 58975
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   SUBJECT_ID        58976 non-null  int64 
 1   HADM_ID           58976 non-null  int64 
 2   CPT_CD            44148 non-null  object
 3   DIAG_ICD9_CODE    58976 non-null  object
 4   DRG_CODE          58890 non-null  object
 5   PROCED_ICD9_CODE  52243 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.1+ MB


In [72]:
#merge in the previously merged dataframe (cpt, diagnosis, procedure, drg codes) with the main_df
main_df = main_df.merge(merged_df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
main_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,HAS_CHARTEVENTS_DATA,READMISSION,GENDER,DOB,AGE,TEXT,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE
0,53486,82574,118464,2100-06-07 19:59:00,2100-06-09,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,...,1,0,M,2044-04-23,56,Admission Date: [**2100-6-7**] D...,"99232, 99291, 99233","311, 6826, 7804, 1991, 3051, 4400, 2720, 28529...","2072, 2073, 314",9229
1,35574,29156,161773,2100-06-09 01:39:00,2100-06-19,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,...,1,0,M,2028-04-23,72,Admission Date: [**2100-6-9**] D...,"99291, 99239, 99253, 94002, 99292, 94003, 99233","51881, 48241, 2859, 4821, 5990, 53081, 2639, 0...","1304, 565","9604, 966, 9672, 3893, 9915"
2,14753,12001,173927,2100-06-14 04:55:00,2100-06-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,1,0,F,2028-10-27,71,Admission Date: [**2100-6-14**] Dischar...,NaN,"7070, 4019, 29410, 431, 2720, 725, 5990, 3310",14,"4513, 966, 4311"
3,25723,21081,159656,2100-06-14 12:02:00,2100-06-17,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,...,1,0,F,2067-04-01,33,Admission Date: [**2100-6-14**] Discharge...,"99233, 99291","51881, E8500, 7863, 30470, 042, 5070, 9708, 49...",87,NaN
4,38932,32096,158366,2100-06-22 03:04:00,2100-06-30,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,...,1,0,F,2070-12-13,29,Admission Date: [**2100-6-22**] ...,"99254, 27252, 27762, 27228","V4586, 27801, 8910, 8074, 8240, 8080, 86121, 3...","9123, 486","7906, 7939, 7909, 390, 9904, 8659, 3893"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45586,15999,13033,107751,2187-03-01 04:32:00,2187-03-05,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,...,1,1,M,2148-04-23,38,Admission Date: [**2187-3-1**] D...,"99291, 99231, 99238, 99232, 99251, 90935","5363, 41401, 412, V4582, 3371, 25063, 5856, 40301","682, 4602, 4603","3995, 3895"
45587,16000,13033,161468,2187-04-05 16:22:00,2187-04-13,EMERGENCY,EMERGENCY ROOM ADMIT,LEFT AGAINST MEDICAL ADVI,Medicare,ENGL,...,1,1,M,2148-04-23,38,Admission Date: [**2187-4-5**] D...,"99291, 99239, 99253, 99231, 99254, 99232, 9093...","99673, 5363, 04111, 41401, 7907, 412, V4582, 3...","682, 4603","3995, 3895"
45588,16001,13033,132307,2187-07-11 07:19:00,2187-07-14,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,...,1,1,M,2148-04-23,39,Admission Date: [**2187-7-11**] ...,"99291, 99231, 99238, 99232, 90935, 99233, 99252","25043, 5363, 41401, 28521, 7907, 40391, 25063,...","73, 483",3995
45589,16002,13033,183332,2187-07-24 13:29:00,2187-07-27,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,...,1,1,M,2148-04-23,39,Admission Date: [**2187-7-24**] ...,"99291, 99238, 99232, 90935, 99233","5363, 41401, 30503, V1582, 40391, 412, V1251, ...","682, 4602, 4603","3995, 3893"


No rows were lost from the main_df (admission and notes combined) after merging with the merged_df (CPT, diagnosis, procedure, DRG codes).

In [73]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45591 entries, 0 to 45590
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                45591 non-null  int64         
 1   SUBJECT_ID            45591 non-null  int64         
 2   HADM_ID               45591 non-null  int64         
 3   ADMITTIME             45591 non-null  object        
 4   DISCHTIME             45591 non-null  datetime64[ns]
 5   ADMISSION_TYPE        45591 non-null  object        
 6   ADMISSION_LOCATION    45591 non-null  object        
 7   DISCHARGE_LOCATION    45591 non-null  object        
 8   INSURANCE             45591 non-null  object        
 9   LANGUAGE              27148 non-null  object        
 10  RELIGION              45190 non-null  object        
 11  MARITAL_STATUS        39968 non-null  object        
 12  ETHNICITY             45591 non-null  object        
 13  DIAGNOSIS       

In [74]:
main_df['READMISSION'].value_counts()

0    33676
1    11915
Name: READMISSION, dtype: int64

There seems to be some class imbalance between readmissions and no readmissions, but it does not seem to be very high.

## Dropping Columns and Handling Missing Values

In [75]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45591 entries, 0 to 45590
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                45591 non-null  int64         
 1   SUBJECT_ID            45591 non-null  int64         
 2   HADM_ID               45591 non-null  int64         
 3   ADMITTIME             45591 non-null  object        
 4   DISCHTIME             45591 non-null  datetime64[ns]
 5   ADMISSION_TYPE        45591 non-null  object        
 6   ADMISSION_LOCATION    45591 non-null  object        
 7   DISCHARGE_LOCATION    45591 non-null  object        
 8   INSURANCE             45591 non-null  object        
 9   LANGUAGE              27148 non-null  object        
 10  RELIGION              45190 non-null  object        
 11  MARITAL_STATUS        39968 non-null  object        
 12  ETHNICITY             45591 non-null  object        
 13  DIAGNOSIS       

In [76]:
#drop any irrelevant columns
columns_to_drop = ['ROW_ID','SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','HAS_CHARTEVENTS_DATA', 'DOB']
main_df.drop(columns_to_drop, axis=1, inplace=True)

There are missing values in the following columns: LANGUAGE, RELIGION, MARITAL STATUS, DIAGNOSIS, CPT_CD, DRG_CODE, and PROCED_ICD9_CODE. Given that all these columns are categorical columns, I decided to fill all the nulls with "Unknown".

In [77]:
main_df.fillna("UNKNOWN", inplace=True)

In [78]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45591 entries, 0 to 45590
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ADMISSION_TYPE      45591 non-null  object
 1   ADMISSION_LOCATION  45591 non-null  object
 2   DISCHARGE_LOCATION  45591 non-null  object
 3   INSURANCE           45591 non-null  object
 4   LANGUAGE            45591 non-null  object
 5   RELIGION            45591 non-null  object
 6   MARITAL_STATUS      45591 non-null  object
 7   ETHNICITY           45591 non-null  object
 8   DIAGNOSIS           45591 non-null  object
 9   READMISSION         45591 non-null  int64 
 10  GENDER              45591 non-null  object
 11  AGE                 45591 non-null  int64 
 12  TEXT                45591 non-null  object
 13  CPT_CD              45591 non-null  object
 14  DIAG_ICD9_CODE      45591 non-null  object
 15  DRG_CODE            45591 non-null  object
 16  PROCED_ICD9_CODE    45

In [79]:
main_df.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,READMISSION,GENDER,AGE,TEXT,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE
0,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,CATHOLIC,SINGLE,OTHER,HYPOTENSION,0,M,56,Admission Date: [**2100-6-7**] D...,"99232, 99291, 99233","311, 6826, 7804, 1991, 3051, 4400, 2720, 28529...","2072, 2073, 314",9229
1,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,UNOBTAINABLE,MARRIED,WHITE,RESPIRATORY DISTRESS,0,M,72,Admission Date: [**2100-6-9**] D...,"99291, 99239, 99253, 94002, 99292, 94003, 99233","51881, 48241, 2859, 4821, 5990, 53081, 2639, 0...","1304, 565","9604, 966, 9672, 3893, 9915"
2,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,UNKNOWN,CATHOLIC,MARRIED,WHITE,RT BASAL GANGLIA BLEEDING,0,F,71,Admission Date: [**2100-6-14**] Dischar...,UNKNOWN,"7070, 4019, 29410, 431, 2720, 725, 5990, 3310",14,"4513, 966, 4311"
3,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,UNKNOWN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,SHORTNESS OF BREATH,0,F,33,Admission Date: [**2100-6-14**] Discharge...,"99233, 99291","51881, E8500, 7863, 30470, 042, 5070, 9708, 49...",87,UNKNOWN
4,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,"STERNAL FRACTURE,RIB FRACTURE,LACERATIONS",0,F,29,Admission Date: [**2100-6-22**] ...,"99254, 27252, 27762, 27228","V4586, 27801, 8910, 8074, 8240, 8080, 86121, 3...","9123, 486","7906, 7939, 7909, 390, 9904, 8659, 3893"


## Export Cleaned and Merged Dataset

Export the cleaned and merged dataset into the data folder. I am planning to preprocess the text data in the modeling notebook instead to adjust the preprocessing step to see how it affects the model performance.

In [80]:
#export the main dataframe to the data folder for analysis in the modeling notebook
main_df.to_csv("data/admissions_cleaned.csv" , index = False)